## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-16-12-11 +0000,nyt,Democrats Counter Trump’s Proposal to Ban Wall...,https://www.nytimes.com/2026/02/24/us/politics...
1,2026-02-24-16-11-09 +0000,nypost,Warner Bros. weighing revised bid from Paramou...,https://nypost.com/2026/02/24/media/warner-bro...
2,2026-02-24-16-09-37 +0000,nyt,"Racing to Catch up With Nvidia, AMD Signs Chip...",https://www.nytimes.com/2026/02/24/business/me...
3,2026-02-24-16-05-58 +0000,bbc,GPs told to guarantee same-day appointments fo...,https://www.bbc.com/news/articles/cm28vp0pkzro...
4,2026-02-24-16-03-05 +0000,nyt,Trump to Address a Complacent Congress Badly S...,https://www.nytimes.com/2026/02/24/us/congress...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2540/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,58
451,mexico,25
135,new,23
98,union,21
514,cartel,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...,162
222,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,161
159,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,142
277,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...,140
95,2026-02-24-13-00-00 +0000,wsj,Dozens of congressional Democrats are preparin...,https://www.wsj.com/politics/policy/democrats-...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,162,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...
265,122,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...
394,94,2026-02-23-17-28-59 +0000,nypost,Ex-British Ambassador to US Peter Mandelson ar...,https://nypost.com/2026/02/23/world-news/ex-br...
233,67,2026-02-24-01-03-34 +0000,nypost,Mexico beach named best in the world — right b...,https://nypost.com/2026/02/23/world-news/mexic...
291,57,2026-02-23-22-17-00 +0000,wsj,Rep. Tony Gonzales of Texas faced calls from R...,https://www.wsj.com/politics/policy/tony-gonza...
159,54,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...
141,38,2026-02-24-11-00-00 +0000,latimes,"Chabria: At San Quentin, Newsom shows off the ...",https://www.latimes.com/california/story/2026-...
329,37,2026-02-23-20-38-35 +0000,latimes,CBS News cuts ties with longevity expert Peter...,https://www.latimes.com/entertainment-arts/bus...
174,37,2026-02-24-09-00-00 +0000,cbc,'Putin will not stop': Zelenskyy warns world o...,https://www.cbc.ca/news/world/zelenskyy-trump-...
44,36,2026-02-24-15-05-00 +0000,wsj,Guthrie Family Offers $1 Million Reward for Na...,https://www.wsj.com/us-news/guthrie-family-off...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
